<a href="https://colab.research.google.com/github/Maoko81/test/blob/main/newGrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean

# Load data
pwt101 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt101.loc[pwt101['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['alpha'] = 1 - data['labsh']  

# Order by year
data = data.sort_values('year')

# Group by countrycode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  
data['k'] = (grouped_data['k_pc'].diff() * 100)  
data['a'] = data['g']-data['alpha']*data['k']

data['g1']=data['g']/100+1
data['k1']=data['k']/100+1
data['a1']=data['a']/100+1

#Remove missing values
data = data.dropna()

#Calculate geometric mean of growth rates for each country
data_fra=data[data['countrycode']=="FRA"]
g_fra=(gmean(data_fra['g1'])-1)*100
k_fra=(gmean(data_fra['k1'])-1)*100
a_fra=(gmean(data_fra['a1'])-1)*100
cap_fra=k_fra*np.mean(data_fra['alpha'])

data_deu=data[data['countrycode']=="DEU"]
g_deu=(gmean(data_deu['g1'])-1)*100
k_deu=(gmean(data_deu['k1'])-1)*100
a_deu=(gmean(data_deu['a1'])-1)*100
cap_deu=k_deu*np.mean(data_deu['alpha'])

data_can=data[data['countrycode']=="CAN"]
g_can=(gmean(data_can['g1'])-1)*100
k_can=(gmean(data_can['k1'])-1)*100
a_can=(gmean(data_can['a1'])-1)*100
cap_can=k_can*np.mean(data_can['alpha'])

data_ita=data[data['countrycode']=="ITA"]
g_ita=(gmean(data_ita['g1'])-1)*100
k_ita=(gmean(data_ita['k1'])-1)*100
a_ita=(gmean(data_ita['a1'])-1)*100
cap_ita=k_ita*np.mean(data_ita['alpha'])

data_jpn=data[data['countrycode']=="JPN"]
g_jpn=(gmean(data_jpn['g1'])-1)*100
k_jpn=(gmean(data_jpn['k1'])-1)*100
a_jpn=(gmean(data_jpn['a1'])-1)*100
cap_jpn=k_jpn*np.mean(data_jpn['alpha'])

data_gbr=data[data['countrycode']=="GBR"]
g_gbr=(gmean(data_gbr['g1'])-1)*100
k_gbr=(gmean(data_gbr['k1'])-1)*100
a_gbr=(gmean(data_gbr['a1'])-1)*100
cap_gbr=k_gbr*np.mean(data_gbr['alpha'])

data_usa=data[data['countrycode']=="USA"]
g_usa=(gmean(data_usa['g1'])-1)*100
k_usa=(gmean(data_usa['k1'])-1)*100
a_usa=(gmean(data_usa['a1'])-1)*100
cap_usa=k_usa*np.mean(data_usa['alpha'])

index=["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"]
list1=[g_fra, a_fra, cap_fra]
list2=[g_deu, a_deu, cap_deu]
list3=[g_can, a_can, cap_can]
list4=[g_ita, a_ita, cap_ita]
list5=[g_jpn, a_jpn, cap_jpn]
list6=[g_gbr, a_gbr, cap_gbr]
list7=[g_usa, a_usa, cap_usa]
lists=[list1,list2,list3,list4,list5,list6,list7]
df1=pd.DataFrame(lists,columns=["Growth Rate","TFP Growth","Capital Deepning"],index=index)
df1["TFP Share"]=df1["TFP Growth"]/df1["Growth Rate"]
df1["Capital Share"]=df1["Capital Deepning"]/df1["Growth Rate"]
df1.loc['Average']=df1.mean()
print(df1)

                Growth Rate  TFP Growth  Capital Deepning  TFP Share  \
France             0.825846    0.256336          0.568151   0.310392   
Germany            0.645232    0.208366          0.442824   0.322932   
Canada             0.882518    0.248086          0.634322   0.281112   
Italy             -0.046486   -0.580573          0.532505  12.489293   
Japan              0.696862    0.066614          0.642975   0.095592   
United Kingdom     1.007664    0.434899          0.568622   0.431592   
United States      1.533139    0.753272          0.789478   0.491326   
Average            0.792111    0.198143          0.596983   2.060320   

                Capital Share  
France               0.687962  
Germany              0.686302  
Canada               0.718764  
Italy              -11.455251  
Japan                0.922673  
United Kingdom       0.564297  
United States        0.514942  
Average             -1.051473  
